In [3]:
import pandas as pd
import numpy as np
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [1]:
import json
import pickle

json_file_path = '../../data_set/yelp_subset_review.json'  # your input file
pickle_file_path = '../../data_set/yelp_subset_review.pkl'  # your output file

data = []

# Read JSON lines
with open(json_file_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

# Save as pickle
with open(pickle_file_path, 'wb') as f:
    pickle.dump(data, f)

In [15]:
# Chargement des données
df = pd.read_pickle('../../data_set/yelp_subset_review.pkl')

In [ ]:
# Conversion des étoiles en classes (négatif, neutre, positif)
def convert_stars_to_sentiment(stars):
    if stars == 1:
        return 0  # Négatif
    elif stars == 3:
        return 1  # Neutre
    else:
        return 2  # Positif

df['labels'] = df['stars'].apply(convert_stars_to_sentiment)


TypeError: list indices must be integers or slices, not str

In [6]:
# Division des données
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['labels'])

# Conversion en datasets Hugging Face
train_dataset = Dataset.from_pandas(train_df[['text', 'labels']])
test_dataset = Dataset.from_pandas(test_df[['text', 'labels']])

In [7]:
# Chargement du tokenizer et du modèle
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Tokenization des données
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2092359 [00:00<?, ? examples/s]

Map:   0%|          | 0/523090 [00:00<?, ? examples/s]

In [8]:
# Fonction de métrique d'évaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [9]:
# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Création du trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

# Entraînement du modèle
trainer.train()

/home/aurelien/jupyter/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Évaluation du modèle
eval_results = trainer.evaluate()
print(f"Résultats d'évaluation: {eval_results}")

# Sauvegarder le modèle
model_path = "./yelp_sentiment_model"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# Exemple d'utilisation
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    sentiment_map = {0: "Négatif", 1: "Neutre", 2: "Positif"}
    return sentiment_map[prediction]

# Test avec quelques exemples
test_reviews = [
    "Le service était horrible et la nourriture froide.",
    "Le restaurant était correct, rien d'extraordinaire.",
    "Une expérience incroyable ! La meilleure nourriture que j'ai jamais goûtée."
]

for review in test_reviews:
    print(f"Review: {review}")
    print(f"Sentiment: {predict_sentiment(review)}\n")